# Usage

To explain the usage of ``docking_py``, we will use a redocking procedure

The same project should be launched from the ``docking`` conda environment:

```
$ conda activate docking
```

To use Docking Python in a project:

In [1]:
from pdb_manip_py import pdb_manip

## Extract Ligand coordinates with pdb_manip_py

First you need to extract the ligand coordinates, we will use the ``1hsg.pdb``
PDB file and extract the coordinates of L-735,524 an inhibitor of the HIV proteases (resname ``MK1``) using the ``pdb_manip_py`` library (Installed with ``docking_py``):

In [2]:
# Create a Coor object
coor_1hsg = pdb_manip.Coor()
coor_1hsg.get_PDB('1hsg', 'data/1hsg.pdb')

In [3]:
# Select res_name MK1
lig_coor = coor_1hsg.select_part_dict(selec_dict={'res_name': ['MK1']})
# Save the ligand coordinates
lig_coor.write_pdb('data/lig.pdb')

In [17]:
view_lig = lig_coor.view
view_lig

NGLWidget()

In [19]:
# To hide this cell add the following info in the cell metadata:
# "nbsphinx": "hidden"
import nglview as nv
nv.write_html('../_static/lig.html', [view_lig])
from IPython.display import IFrame

In [20]:
# Unecessary, only need to nglview online:
IFrame(src='../_static/lig.html', width=800, height=300)

## Extract Receptor coordinates with pdb_manip_py

Then you need to extract the receptor coordinates, we will use the 
``1hsg.pdb`` PDB file and extract the coordinates of the HIV II protease 
using the ``pdb_manip_py`` library:

In [21]:
# Keep only the amino acids
rec_coor = coor_1hsg.select_part_dict(selec_dict={'res_name': pdb_manip.PROTEIN_RES})
rec_coor.write_pdb('data/rec.pdb')

In [22]:
view_rec = rec_coor.view
view_rec

NGLWidget()

In [23]:
# To hide this cell add the following info in the cell metadata:
# "nbsphinx": "hidden"
import nglview as nv
nv.write_html('../_static/rec.html', [view_rec])
from IPython.display import IFrame

In [25]:
# Unecessary, only need to nglview online:
IFrame(src='../_static/rec.html', width=800, height=300)

## Prepare Ligand and receptor structures

You need to create a ``Docking`` object, and the use the
functions ``prepare_ligand()`` and ``prepare_receptor()``:

In [8]:
from docking_py import docking

test_dock = docking.Docking('test', lig_pdb='data/lig.pdb', rec_pdb='data/rec.pdb')
test_dock.prepare_ligand()

python2.5 ../../../../../../miniconda3/envs/docking/bin/prepare_ligand4.py -l lig.pdb -B none -A hydrogens -o lig.pdbqt


In [9]:
test_dock.prepare_receptor()

python2.5 ../../../../../miniconda3/envs/docking/bin/prepare_receptor4.py -r data/rec.pdb -A checkhydrogens -o data/rec.pdbqt


## Launch docking calculation

Launch the docking:

In [10]:
test_dock.run_docking(out_pdb='test_dock.pdb',
                      num_modes=10,
                      energy_range=10,
                      exhaustiveness=16,
                      dock_bin='smina')

Grid points: None


smina --ligand data/lig.pdbqt --receptor data/rec.pdbqt --log test_dock_log.txt --num_modes 10 --exhaustiveness 16 --energy_range 10 --out test_dock.pdb --size_x 66.00 --size_y 81.00 --size_z 83.00 --center_x 16.07 --center_y 26.49 --center_z 3.77


## Analysis

Extract affinity and RMSD to crystal structure:

In [11]:
rmsd_list = test_dock.compute_dock_rmsd(test_dock.lig_pdbqt)

File name doesn't finish with .pdb read it as .pdb anyway


In [12]:
rmsd_list

[0.6172348337545442,
 4.523207300135602,
 11.579705736330263,
 9.904196947759067,
 10.692842899809198,
 10.975378963844483,
 12.19258827074875,
 10.207969165313932,
 9.394261151362569,
 12.029979500398163]

In [26]:
view_dock = test_dock.view_dock(ref_pdb="data/1hsg.pdb")
view_dock

NGLWidget(max_frame=9)

In [29]:
# To hide this cell add the following info in the cell metadata:
# "nbsphinx": "hidden"
import nglview as nv
nv.write_html('../_static/dock.html', [view_dock], frame_range=(0, 10))
from IPython.display import IFrame

In [30]:
# Unecessary, only need to nglview online:
IFrame(src='../_static/dock.html', width=800, height=300)

In [15]:
test_dock.affinity

{1: {'affinity': -11.9, 'rmsd_low': 0.0, 'rmsd_high': 0.0},
 2: {'affinity': -10.6, 'rmsd_low': 2.288, 'rmsd_high': 4.387},
 3: {'affinity': -9.3, 'rmsd_low': 3.55, 'rmsd_high': 11.574},
 4: {'affinity': -8.8, 'rmsd_low': 5.812, 'rmsd_high': 9.719},
 5: {'affinity': -8.7, 'rmsd_low': 5.959, 'rmsd_high': 10.368},
 6: {'affinity': -8.7, 'rmsd_low': 3.265, 'rmsd_high': 10.921},
 7: {'affinity': -8.4, 'rmsd_low': 3.702, 'rmsd_high': 12.258},
 8: {'affinity': -8.3, 'rmsd_low': 5.468, 'rmsd_high': 9.968},
 9: {'affinity': -8.2, 'rmsd_low': 5.679, 'rmsd_high': 9.289},
 10: {'affinity': -8.1, 'rmsd_low': 7.058, 'rmsd_high': 11.97}}